## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import time

## Importing the dataset

In [2]:
(X_test, y_test), (X_train, y_train) = cifar10.load_data()
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [30]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_train = y_train.ravel()
y_test = y_test.ravel()

## Feature Scaling

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## PCA

In [12]:
from sklearn.decomposition import PCA
# Compress Image Data - Keep Explained Variance Ratio over 90%
print(X_train.shape) # 3072 values per image 
pca = PCA(n_components = 50)
X_train_comp = pca.fit_transform(X_train)
i = 1
print(pca.explained_variance_ratio_.sum())
while pca.explained_variance_ratio_.sum() < 0.9:
    pca = PCA(n_components = 50 + 10*i)
    i+=1
    X_train_comp = pca.fit_transform(X_train)
    print(pca.explained_variance_ratio_.sum())
X_test_comp = pca.transform(X_test)

(50000, 3072)
0.8382544916919416
0.8551335205608408
0.8689602916833461
0.8802753406627262
0.8900514175469053
0.8983498488779204
0.9055101922217743


In [9]:
print(pca.n_components_)

110


## Training the Kernel SVM model on the Training set

### RBF Model

In [10]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, make_scorer


grid = {
    'C': np.logspace(-2, 3, 5),
    'gamma': np.logspace(-2, 2, 5),
    'kernel':['rbf'],
}
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
svc = SVC()
f1 = make_scorer(f1_score , average='weighted')
gs = GridSearchCV(estimator=tr, cv=kfold, param_grid=grid, scoring=f1, refit=True, verbose=3, n_jobs = -1)

In [11]:
res = gs.fit(X_train_comp, y_train.ravel())

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  7.1min finished


In [46]:
resDataRBF = pd.DataFrame.from_dict(res.cv_results_)
display(resDataRBF.sort_values('rank_test_score'))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
20,40.285992,4.753784,11.558569,0.124605,1000,0.01,rbf,"{'C': 1000.0, 'gamma': 0.01, 'kernel': 'rbf'}",0.123507,0.137109,0.137277,0.132631,0.006452,1
10,42.838982,4.301881,12.542840,0.695694,3.16228,0.01,rbf,"{'C': 3.1622776601683795, 'gamma': 0.01, 'kern...",0.123507,0.137109,0.137277,0.132631,0.006452,1
15,41.548701,5.593333,12.382869,0.941476,56.2341,0.01,rbf,"{'C': 56.23413251903491, 'gamma': 0.01, 'kerne...",0.123507,0.137109,0.137277,0.132631,0.006452,1
7,62.206490,2.297098,11.908363,0.134193,0.177828,1,rbf,"{'C': 0.1778279410038923, 'gamma': 1.0, 'kerne...",0.094250,0.108328,0.072951,0.091843,0.014542,4
12,56.719710,2.360340,13.698418,0.381959,3.16228,1,rbf,"{'C': 3.1622776601683795, 'gamma': 1.0, 'kerne...",0.091570,0.108697,0.073505,0.091257,0.014369,5
22,59.429469,2.435926,10.679411,0.894475,1000,1,rbf,"{'C': 1000.0, 'gamma': 1.0, 'kernel': 'rbf'}",0.091570,0.108697,0.073505,0.091257,0.014369,5
17,60.469444,4.515879,12.026671,0.555342,56.2341,1,rbf,"{'C': 56.23413251903491, 'gamma': 1.0, 'kernel...",0.091570,0.108697,0.073505,0.091257,0.014369,5
2,56.203778,0.858956,13.304187,0.348743,0.01,1,rbf,"{'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'}",0.093163,0.107621,0.071007,0.090597,0.015058,8
1,51.193269,0.992515,13.363480,0.400790,0.01,0.1,rbf,"{'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}",0.052279,0.057217,0.051013,0.053503,0.002677,9
6,51.613881,2.873677,13.117737,0.210893,0.177828,0.1,rbf,"{'C': 0.1778279410038923, 'gamma': 0.1, 'kerne...",0.020548,0.057217,0.051013,0.042926,0.016025,10


### Linear Model

In [13]:
grid2 = {
    'C': np.logspace(-3, 0, 4),
    'kernel':['linear']
}

gs = GridSearchCV(estimator=svc, cv=kfold, param_grid=grid2, scoring=f1, refit=True, verbose=3, n_jobs = -1)
res2 = gs.fit(X_train_comp, y_train.ravel())

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   29.2s remaining:   58.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  2.0min remaining:   39.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 14.0min finished


In [45]:
resDataLin = pd.DataFrame.from_dict(res2.cv_results_)
display(resDataLin)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,17.170924,1.051450,6.007870,0.311989,0.001,linear,"{'C': 0.001, 'kernel': 'linear'}",0.384771,0.395587,0.391921,0.390760,0.004491,1
1,27.037357,0.912513,3.404919,0.014334,0.01,linear,"{'C': 0.01, 'kernel': 'linear'}",0.378979,0.368239,0.384919,0.377379,0.006903,2
2,110.704484,3.613670,2.523926,0.139369,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.376381,0.364685,0.382381,0.374482,0.007348,3
3,824.273685,16.310758,2.223605,0.049069,1,linear,"{'C': 1.0, 'kernel': 'linear'}",0.374289,0.365743,0.380342,0.373458,0.005989,4


### Polynomial Model

In [15]:
grid2 = {
    'C': np.logspace(-2, 3, 5),
    'gamma': np.logspace(-2, 2, 5),
    'kernel':['poly'],
    'degree': [2,3,4]
}

gs = GridSearchCV(estimator=svc, cv=kfold, param_grid=grid2, scoring=f1, refit=True, verbose=3, n_jobs = -1)
res3 = gs.fit(X_train_comp, y_train.ravel())

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed: 16.3min finished


In [44]:
resDataPoly = pd.DataFrame.from_dict(res3.cv_results_)
display(resDataPoly)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,29.592214,1.361123,10.152247,0.335226,0.01,2,0.01,poly,"{'C': 0.01, 'degree': 2, 'gamma': 0.01, 'kerne...",0.386754,0.379517,0.384810,0.383694,0.003058,1
1,36.062276,2.097420,11.807037,0.215707,0.01,2,0.1,poly,"{'C': 0.01, 'degree': 2, 'gamma': 0.1, 'kernel...",0.365876,0.351848,0.365759,0.361161,0.006586,27
2,34.841730,1.015300,11.499178,0.212968,0.01,2,1,poly,"{'C': 0.01, 'degree': 2, 'gamma': 1.0, 'kernel...",0.366829,0.351412,0.365388,0.361210,0.006953,15
3,36.755936,1.395633,11.535931,0.358367,0.01,2,10,poly,"{'C': 0.01, 'degree': 2, 'gamma': 10.0, 'kerne...",0.366829,0.351754,0.365388,0.361324,0.006792,5
4,33.471046,3.370331,9.501217,0.068884,0.01,2,100,poly,"{'C': 0.01, 'degree': 2, 'gamma': 100.0, 'kern...",0.366829,0.351754,0.365388,0.361324,0.006792,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,41.626732,3.063829,10.509835,0.711490,1000,4,0.01,poly,"{'C': 1000.0, 'degree': 4, 'gamma': 0.01, 'ker...",0.312361,0.297902,0.307085,0.305783,0.005974,52
71,40.284027,1.033146,10.627142,0.100400,1000,4,0.1,poly,"{'C': 1000.0, 'degree': 4, 'gamma': 0.1, 'kern...",0.312361,0.297902,0.307085,0.305783,0.005974,52
72,39.655594,0.553503,9.122752,1.626144,1000,4,1,poly,"{'C': 1000.0, 'degree': 4, 'gamma': 1.0, 'kern...",0.312361,0.297902,0.307085,0.305783,0.005974,52
73,36.282132,4.085234,5.493933,0.843550,1000,4,10,poly,"{'C': 1000.0, 'degree': 4, 'gamma': 10.0, 'ker...",0.312361,0.297902,0.307085,0.305783,0.005974,52


## Test Accuracy and Comparison with KNN and Nearest Centroid

In [31]:
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
classifier1 = KNeighborsClassifier(n_neighbors = 1)
classifier2 = KNeighborsClassifier(n_neighbors = 3)
classifier3 = NearestCentroid()
ts = time.time()
classifier1.fit(X_train_comp, y_train_cat)
t1 = time.time() - ts
classifier2.fit(X_train_comp, y_train_cat)
t2 = time.time() - t1 - ts 
classifier3.fit(X_train_comp, y_train)
t3 = time.time() - t2 - t1 - ts

In [38]:
y_pred1 = classifier1.predict(X_test_comp)
y_pred2 = classifier2.predict(X_test_comp)
y_pred3 = classifier3.predict(X_test_comp)
y_pred4 = res.predict(X_test_comp)
y_pred5 = res2.predict(X_test_comp)
y_pred6 = res3.predict(X_test_comp)

In [43]:
from sklearn.metrics import accuracy_score

print("KNN with 1 closest neighbour model accuracy on test set:", accuracy_score(y_test_cat, y_pred1))
print("KNN with 1 closest neighbour model accuracy on training set:", accuracy_score(y_train_cat, classifier1.predict(X_train_comp)))
print("Training time:", f'{t1:.4f}', "seconds\n")
print("KNN with 3 closest neighbours model accuracy on test set:",accuracy_score(y_test_cat, y_pred2))
print("KNN with 3 closest neighbours model accuracy on training set:", accuracy_score(y_train_cat, classifier2.predict(X_train_comp)))
print("Training time:", f'{t2:.4f}', "seconds\n")
print("Nearest Centroid model accuracy on test set:",accuracy_score(y_test, y_pred3))
print("Nearest Centroid model accuracy on training set:",accuracy_score(y_train, classifier3.predict(X_train_comp)))
print("Training time:", f'{t3:.4f}', "seconds\n")
print("RBF SVM model accuracy on test set:", accuracy_score(y_test, y_pred4))
print("RBF SVM model accuracy on training set:", accuracy_score(y_train, res.predict(X_train_comp)))
print("Mean Training time: 42.8389 seconds\n")
print("Linear SVM model accuracy on test set:",accuracy_score(y_test, y_pred5))
print("Linear SVM model accuracy on training set:", accuracy_score(y_train, res2.predict(X_train_comp)))
print("Mean Training time: 17.1709 seconds\n")
print("Polynomial SVM model accuracy on test set:",accuracy_score(y_test, y_pred6))
print("Polynomial SVM model accuracy on training set:",accuracy_score(y_train, res3.predict(X_train_comp)))
print("Mean Training time: 29.5922 seconds")

KNN with 1 closest neighbour model accuracy on test set: 0.30942
KNN with 1 closest neighbour model accuracy on training set: 1.0
Training time: 0.1207 seconds

KNN with 3 closest neighbours model accuracy on test set: 0.2362
KNN with 3 closest neighbours model accuracy on training set: 0.4529
Training time: 0.1005 seconds

Nearest Centroid model accuracy on test set: 0.2731
Nearest Centroid model accuracy on training set: 0.2853
Training time: 0.0074 seconds

RBF SVM model accuracy on test set: 0.17662
RBF SVM model accuracy on training set: 1.0
Mean Training time: 42,8389 seconds

Linear SVM model accuracy on test set: 0.38954
Linear SVM model accuracy on training set: 0.4478
Mean Training time: 17,1709 seconds

Polynomial SVM model accuracy on test set: 0.40488
Polynomial SVM model accuracy on training set: 0.8242
Mean Training time: 29,5922 seconds
